# КУРСОВАЯ РАБОТА ОПТИМИЗАЦИЯ РАСПИСАНИЯ АВТОБУСОВ  Шедова А.Р. БВТ 2203

# Наивный метод

In [51]:
from datetime import datetime, timedelta

class Stop:
    def __init__(self, name):
        self.name = name

class Bus:
    def __init__(self, bus_number):
        self.bus_number = bus_number
        self.current_trip = None

class Driver:
    def __init__(self, name, driver_type, bus, shift_start, shift_end, breaks):
        self.name = name
        self.driver_type = driver_type
        self.bus = bus
        self.shift_start = shift_start
        self.shift_end = shift_end
        self.breaks = breaks
        self.trips = []
        self.loop_count = 0

    def is_available(self, trip_start, trip_duration):
        trip_end = trip_start + trip_duration
        #проверка на рабочее время
        if trip_start < self.shift_start or trip_end > self.shift_end:
            return False
        #проверка на пересечение с перерывами
        for break_start, break_end in self.breaks:
            if trip_start < break_end and trip_end > break_start:
                return False
        #проверка на пересечение с другими рейсами
        for trip in self.trips:
            if trip_start < trip.departure_time + trip.route.duration and trip_end > trip.departure_time:
                return False
        #проверка занятости автобуса
        if self.bus.current_trip:
            bus_trip_end = self.bus.current_trip.departure_time + self.bus.current_trip.route.duration
            if trip_start < bus_trip_end:
                return False
        return True

class Route:
    def __init__(self, stops, direction):
        self.stops = stops
        self.direction = direction
        self.duration = timedelta(minutes=60)

class Trip:
    def __init__(self, departure_time, bus, driver, route):
        self.departure_time = departure_time
        self.bus = bus
        self.driver = driver
        self.route = route
        self.arrival_times = []

class Schedule:
    def __init__(self):
        self.trips = []
    
    def add_trip(self, trip):
        self.trips.append(trip)
    
    def generate_arrival_times(self):
        for trip in self.trips:
            current_time = trip.departure_time
            for stop in trip.route.stops:
                trip.arrival_times.append((stop.name, current_time))
                current_time += timedelta(minutes=20)

stop_names = [
    "Улица маршала Тухачевского",
    "Университет связи и информатики",
    "Метро Октябрьское поле",
    "МЦК Панфиловская"
]
stops = [Stop(name) for name in stop_names]

route_there = Route(stops, "в сторону МЦД Панфиловская")
route_back = Route(stops[::-1], "в сторону улицы Маршала Тухачевского")

buses = [Bus(f"A00{i}AA") for i in range(1, 26)]

DRIVERS_TYPE1 = [
    "Павел", "Петр", "Панкрат", "Прокл", "Платон", 
    "Полина", "Порфирий", "Пересвет", "Пимен", "Прохор", 
    "Потап", "Пиона", "Протас", "Президий", "Преслав"
]

DRIVERS_TYPE2 = [
    "Вадим", "Валерий", "Василий", "Виктор", "Владимир", 
    "Владислав", "Вячеслав", "Виталий", "Валентин", "Венедикт", 
    "Велор", "Вилен", "Витольд", "Валдар", "Варлам",


]

start_date = datetime.strptime("2024-10-01 06:00", "%Y-%m-%d %H:%M")
end_date = datetime.strptime("2024-10-02 03:00", "%Y-%m-%d %H:%M")

shift_start_times = [start_date + timedelta(hours=i) for i in range(int((end_date - start_date).total_seconds() / (60*60)))]


drivers = []

for i, name in enumerate(DRIVERS_TYPE1):
    shift_start = start_date
    shift_end = shift_start + timedelta(hours=8)
    break_start = shift_start + timedelta(hours=4)
    break_end = break_start + timedelta(hours=1)
    breaks = [(break_start, break_end)]
    drivers.append(Driver(name, 1, buses[i % len(buses)], shift_start, shift_end, breaks))

for i, name in enumerate(DRIVERS_TYPE2):
    shift_start = shift_start_times[i % len(shift_start_times)]
    shift_end = shift_start + timedelta(hours=12)
    breaks = []
    break_time = shift_start + timedelta(hours=2, minutes=10)
    while break_time < shift_end:
        break_start = break_time
        break_end = break_start + timedelta(minutes=10)
        breaks.append((break_start, break_end))
        break_time += timedelta(hours=2, minutes=10)
    drivers.append(Driver(name, 2, buses[i % len(buses)], shift_start, shift_end, breaks))

all_drivers = drivers.copy()

start_time = datetime.strptime("2024-10-01 06:00", "%Y-%m-%d %H:%M")
end_time = datetime.strptime("2024-10-02 03:00", "%Y-%m-%d %H:%M")

schedule = Schedule()

def is_peak_time(time):
    return (7 <= time.hour < 9) or (17 <= time.hour < 19)

def generate_trips(schedule, start_time, end_time, available_drivers):
    current_time = start_time
    while current_time < end_time:
        interval = timedelta(minutes=10 if is_peak_time(current_time) else 20)
        for driver in available_drivers:
            on_break = any(
                break_start <= current_time < break_end
                for break_start, break_end in driver.breaks
            )

            if not on_break and driver.is_available(current_time, route_there.duration + route_back.duration):
                selected_bus = driver.bus

                trip_there = Trip(current_time, selected_bus, driver, route_there)
                schedule.add_trip(trip_there)
                driver.trips.append(trip_there)

                trip_back_departure = current_time + route_there.duration
                trip_back = Trip(trip_back_departure, selected_bus, driver, route_back)
                schedule.add_trip(trip_back)
                driver.trips.append(trip_back)

                selected_bus.current_trip = trip_back
                break

        current_time += interval

print(f"Генерация расписания на {start_date.strftime('%Y-%m-%d')} ({start_date.strftime('%A')})")
generate_trips(schedule, start_time, end_time, all_drivers.copy())
schedule.generate_arrival_times()

stop_schedules = {stop.name: {"в сторону МЦД Панфиловская": [], "в сторону улицы Маршала Тухачевского": []} for stop in stops}

for trip in schedule.trips:
    for stop_name, arrival_time in trip.arrival_times:
        direction = trip.route.direction
        stop_schedules[stop_name][direction].append({
            "Время прибытия": arrival_time,
            "Водитель": trip.driver.name,
            "Номер рейса": trip.bus.bus_number,
            "Направление": direction
        })

        
for stop_name, directions in stop_schedules.items():
    print(f"Остановка: {stop_name}")
    for direction in ["в сторону МЦД Панфиловская", "в сторону улицы Маршала Тухачевского"]:
        print(f"  {direction}:")
        sorted_entries = sorted(directions[direction], key=lambda x: x["Время прибытия"])
        for entry in sorted_entries:
            print(f"    Время: {entry['Время прибытия'].strftime('%H:%M')}, Водитель: {entry['Водитель']}, Номер рейса: {entry['Номер рейса']}")

print(f"Всего сгенерированных рейсов: {len(schedule.trips)}")

def print_driver_schedule(driver):
    events = []
    #начало и конец смены
    events.append({'time': driver.shift_start, 'description': 'Начало смены'})
    events.append({'time': driver.shift_end, 'description': 'Конец смены'})
    #все перерывы
    for break_start, break_end in driver.breaks:
        events.append({'time': break_start, 'description': 'Начало перерыва'})
        events.append({'time': break_end, 'description': 'Конец перерыва'})
    #все выезды
    for trip in driver.trips:
        events.append({
            'time': trip.departure_time,
            'description': f'Выезд в "{trip.route.direction}" на автобусе {trip.bus.bus_number}'
        })
    events.sort(key=lambda x: x['time'])
    
    print(f"Расписание для водителя {driver.name}:")
    for event in events:
        print(f"  {event['time'].strftime('%Y-%m-%d %H:%M')}: {event['description']}")

driver_names = [driver.name for driver in all_drivers]
driver_dict = {driver.name: driver for driver in all_drivers}


while True:
    print("Список водителей:")
    print(", ".join(driver_names))
    input_name = input("Введите имя водителя: ").strip()
    
    if input_name in driver_dict:
        selected_driver = driver_dict[input_name]
        break
    else:
        print("Водитель с таким именем не найден. Пожалуйста, введите корректное имя.")

print_driver_schedule(selected_driver)

def print_driver_status_at_hour(target_time, drivers):
    working_drivers = []
    on_break_drivers = []
    off_duty_drivers = []
    
    for driver in drivers:
        if driver.shift_start <= target_time < driver.shift_end:
            on_break = any(break_start <= target_time < break_end for break_start, break_end in driver.breaks)
            if on_break:
                on_break_drivers.append(driver.name)
            else:
                working_drivers.append(driver.name)
        else:
            off_duty_drivers.append(driver.name)
    
    print(f"Статус на {target_time.strftime('%Y-%m-%d %H:%M')}:\n")
    print("Водители на работе:")
    for name in working_drivers:
        print(f" - {name}")
    print("\nВодители в перерыве:")
    for name in on_break_drivers:
        print(f" - {name}")
    print("\nВодители не на смене:")
    for name in off_duty_drivers:
        print(f" - {name}")



print("\nХотите увидеть статус водителей в конкретное время? (да/нет)")
choice = input().strip().lower()

if choice == 'да':
    while True:
        time_input = input("Введите время в формате 'ГГГГ-ММ-ДД ЧЧ:ММ': ")
        try:
            target_time = datetime.strptime(time_input, "%Y-%m-%d %H:%M")
            break
        except ValueError:
            print("Неверный формат времени. Пожалуйста, попробуйте снова.")
    print_driver_status_at_hour(target_time, all_drivers)
else:
    print("Выход из программы.")

Генерация расписания на 2024-10-01 (Tuesday)
Остановка: Улица маршала Тухачевского
  в сторону МЦД Панфиловская:
    Время: 06:00, Водитель: Павел, Номер рейса: A001AA
    Время: 06:20, Водитель: Петр, Номер рейса: A002AA
    Время: 06:40, Водитель: Панкрат, Номер рейса: A003AA
    Время: 07:00, Водитель: Прокл, Номер рейса: A004AA
    Время: 07:10, Водитель: Платон, Номер рейса: A005AA
    Время: 07:20, Водитель: Полина, Номер рейса: A006AA
    Время: 07:30, Водитель: Порфирий, Номер рейса: A007AA
    Время: 07:40, Водитель: Пересвет, Номер рейса: A008AA
    Время: 07:50, Водитель: Пимен, Номер рейса: A009AA
    Время: 08:00, Водитель: Павел, Номер рейса: A001AA
    Время: 09:00, Водитель: Виктор, Номер рейса: A004AA
    Время: 09:20, Водитель: Валерий, Номер рейса: A002AA
    Время: 10:00, Водитель: Владимир, Номер рейса: A005AA
    Время: 10:20, Водитель: Василий, Номер рейса: A003AA
    Время: 11:00, Водитель: Павел, Номер рейса: A001AA
    Время: 11:20, Водитель: Петр, Номер рейса

# Генетический алгоритм

In [ ]:
datetime import datetime, timedelta
import random

class Stop:
    def __init__(self, name):
        self.name = name

class Bus:
    def __init__(self, bus_number):
        self.bus_number = bus_number
        self.current_trip = None

class Driver:
    def __init__(self, name, driver_type, bus, shift_start, shift_end, breaks):
        self.name = name
        self.driver_type = driver_type
        self.bus = bus
        self.shift_start = shift_start
        self.shift_end = shift_end
        self.breaks = breaks
        self.trips = []

    def is_available(self, trip_start, trip_duration):
        trip_end = trip_start + trip_duration
        if trip_start < self.shift_start or trip_end > self.shift_end:
            return False
        for break_start, break_end in self.breaks:
            if trip_start < break_end and trip_end > break_start:
                return False
        for trip in self.trips:
            if trip_start < trip.departure_time + trip.route.duration and trip_end > trip.departure_time:
                return False
        if self.bus.current_trip:
            bus_trip_end = self.bus.current_trip.departure_time + self.bus.current_trip.route.duration
            if trip_start < bus_trip_end:
                return False
        return True

class Route:
    def __init__(self, stops, direction):
        self.stops = stops
        self.direction = direction
        self.duration = timedelta(minutes=60)

class Trip:
    def __init__(self, departure_time, bus, driver, route):
        self.departure_time = departure_time
        self.bus = bus
        self.driver = driver
        self.route = route
        self.arrival_times = []

class Schedule:
    def __init__(self):
        self.trips = []
    
    def add_trip(self, trip):
        self.trips.append(trip)
    
    def generate_arrival_times(self):
        for trip in self.trips:
            current_time = trip.departure_time
            for stop in trip.route.stops:
                trip.arrival_times.append((stop.name, current_time))
                current_time += timedelta(minutes=20)


stop_names = [
    "Улица маршала Тухачевского",
    "Университет связи и информатики",
    "Метро Октябрьское поле",
    "МЦК Панфиловская"
]
stops = [Stop(name) for name in stop_names]

route_there = Route(stops, "в сторону МЦД Панфиловская")
route_back = Route(stops[::-1], "в сторону улицы Маршала Тухачевского")

buses = [Bus(f"A00{i}AA") for i in range(1, 31)]

DRIVERS_TYPE1 = [
    "Павел", "Петр", "Панкрат", "Прокл", "Платон", 
    "Полина", "Порфирий", "Пересвет", "Пимен", "Прохор", 
    "Потап", "Пиона", "Протас", "Президий", "Преслав",
    "Платини", "Праксис", "Поликарп", "Палладий", "Парамон",
    
]

DRIVERS_TYPE2 = [
    "Вадим", "Валерий", "Василий", "Виктор", "Владимир", 
    "Владислав", "Вячеслав", "Виталий", "Валентин", "Венедикт", 
    "Велор", "Вилен", "Витольд", "Валдар", "Варлам",
    "Варфоломей", "Василевс", "Велимир", "Вероний", "Вильгельм"
]

start_date = datetime.strptime("2024-10-01 06:00", "%Y-%m-%d %H:%M")
end_date = datetime.strptime("2024-10-02 03:00", "%Y-%m-%d %H:%M")

drivers_type1 = []
for i, name in enumerate(DRIVERS_TYPE1):
    shift_start = start_date + timedelta(hours=i)
    shift_end = min(shift_start + timedelta(hours=8), end_date)
    break_start = shift_start + timedelta(hours=4)
    break_end = break_start + timedelta(hours=1)
    breaks = [(break_start, break_end)]
    drivers_type1.append(Driver(name, 1, buses[i % len(buses)], shift_start, shift_end, breaks))

drivers_type2 = []
shift_start_times = [start_date + timedelta(hours=i) for i in range(24)]
for i, name in enumerate(DRIVERS_TYPE2):
    shift_start = shift_start_times[i % len(shift_start_times)]
    shift_end = min(shift_start + timedelta(hours=12), end_date)
    breaks = []
    break_time = shift_start + timedelta(hours=2, minutes=10)
    while break_time < shift_end:
        break_start = break_time
        break_end = break_start + timedelta(minutes=10)
        breaks.append((break_start, break_end))
        break_time += timedelta(hours=2, minutes=10)
    drivers_type2.append(Driver(name, 2, buses[i % len(buses)], shift_start, shift_end, breaks))

all_drivers = drivers_type1 + drivers_type2

def get_interval(current_time):
    if (current_time.hour >= 7 and current_time.hour < 9) or (current_time.hour >= 18 and current_time.hour < 20):
        return timedelta(minutes=5)
    else:
        return timedelta(minutes=10)

trips = []
current_time = start_date
while current_time < end_date:
    interval = get_interval(current_time)
    route = route_there if len(trips) % 2 == 0 else route_back
    trip = Trip(current_time, random.choice(buses), random.choice(all_drivers), route)
    trips.append(trip)
    current_time += interval


def generate_initial_population(pop_size, trips, drivers, buses):
    population = []
    for _ in range(pop_size):
        chromosome = []
        for trip in trips:
            available_drivers = [d for d in drivers if d.is_available(trip.departure_time, trip.route.duration)]
            available_buses = [b for b in buses if not b.current_trip or b.current_trip.departure_time + b.current_trip.route.duration <= trip.departure_time]
            if available_drivers and available_buses:
                driver = random.choice(available_drivers)
                bus = random.choice(available_buses)
                chromosome.append((trip, driver, bus))
            else:
                continue
        population.append(chromosome)
    return population

def fitness_function(chromosome, start_time, end_time):
    fitness = 0
    driver_trips = {}
    bus_trips = {}
    for trip, driver, bus in chromosome:
        if start_time <= trip.departure_time < end_time:
            if driver.is_available(trip.departure_time, trip.route.duration):
                if driver.name not in driver_trips:
                    driver_trips[driver.name] = []
                driver_trips[driver.name].append((trip.departure_time, trip.route.duration))
                if bus.bus_number not in bus_trips:
                    bus_trips[bus.bus_number] = []
                bus_trips[bus.bus_number].append((trip.departure_time, trip.route.duration))
                fitness += 1
            else:
                fitness -= 10  
        else:
            fitness -= 100  
    for trips_list in driver_trips.values():
        for i in range(len(trips_list)):
            for j in range(i+1, len(trips_list)):
                if trips_list[i][0] < trips_list[j][0] + trips_list[j][1] and trips_list[i][0] + trips_list[i][1] > trips_list[j][0]:
                    fitness -= 10  
    for trips_list in bus_trips.values():
        for i in range(len(trips_list)):
            for j in range(i+1, len(trips_list)):
                if trips_list[i][0] < trips_list[j][0] + trips_list[j][1] and trips_list[i][0] + trips_list[i][1] > trips_list[j][0]:
                    fitness -= 10  
    return fitness

def selection(population, fitness_scores, num_parents):
    parents = []
    for _ in range(num_parents):
        max_fitness_idx = fitness_scores.index(max(fitness_scores))
        parents.append(population[max_fitness_idx])
        fitness_scores[max_fitness_idx] = -9999
    return parents

def crossover(parent1, parent2):
    crossover_point = random.randint(0, len(parent1)-1)
    child = parent1[:crossover_point] + parent2[crossover_point:]
    # Ensure no duplicate assignments for drivers and buses
    driver_assigned = {}
    bus_assigned = {}
    for i, (trip, driver, bus) in enumerate(child):
        if driver.name in driver_assigned:
            available_drivers = [d for d in all_drivers if d.is_available(trip.departure_time, trip.route.duration)]
            if available_drivers:
                child[i] = (trip, random.choice(available_drivers), bus)
            else:
                child[i] = (trip, None, bus)
        else:
            driver_assigned[driver.name] = True
        if bus.bus_number in bus_assigned:
            available_buses = [b for b in buses if not b.current_trip or b.current_trip.departure_time + b.current_trip.route.duration <= trip.departure_time]
            if available_buses:
                child[i] = (trip, driver, random.choice(available_buses))
            else:
                child[i] = (trip, driver, None)
        else:
            bus_assigned[bus.bus_number] = True
    return child

def mutation(child, trips, drivers, buses):
    mutation_index = random.randint(0, len(child)-1)
    trip, driver, bus = child[mutation_index]
    available_drivers = [d for d in drivers if d.is_available(trip.departure_time, trip.route.duration)]
    available_buses = [b for b in buses if not b.current_trip or b.current_trip.departure_time + b.current_trip.route.duration <= trip.departure_time]
    if available_drivers and available_buses:
        new_driver = random.choice(available_drivers)
        new_bus = random.choice(available_buses)
        child[mutation_index] = (trip, new_driver, new_bus)
    return child

pop_size = 400
num_generations = 300
num_parents = 80

population = generate_initial_population(pop_size, trips, all_drivers, buses)

for generation in range(num_generations):
    fitness_scores = [fitness_function(chromosome, start_date, end_date) for chromosome in population]
    parents = selection(population, fitness_scores, num_parents)
    children = []
    for _ in range(pop_size - num_parents):
        parent1 = random.choice(parents)
        parent2 = random.choice(parents)
        child = crossover(parent1, parent2)
        child = mutation(child, trips, all_drivers, buses)
        children.append(child)
    population = parents + children

best_chromosome = population[fitness_scores.index(max(fitness_scores))]
schedule = Schedule()
for trip, driver, bus in best_chromosome:
    schedule.add_trip(trip)
    driver.trips.append(trip)
    bus.current_trip = trip

schedule.generate_arrival_times()

for trip in schedule.trips:
    if not (start_date <= trip.departure_time < end_date):
        print(f"Trip out of bounds: {trip.departure_time}")

stop_schedules = {stop.name: {"в сторону МЦД Панфиловская": [], "в сторону улицы Маршала Тухачевского": []} for stop in stops}

for trip in schedule.trips:
    for stop_name, arrival_time in trip.arrival_times:
        direction = trip.route.direction
        stop_schedules[stop_name][direction].append({
            "Время прибытия": arrival_time,
            "Водитель": trip.driver.name,
            "Номер рейса": trip.bus.bus_number,
            "Направление": direction
        })

for stop_name, directions in stop_schedules.items():
    print(f"Остановка: {stop_name}")
    for direction in ["в сторону МЦД Панфиловская", "в сторону улицы Маршала Тухачевского"]:
        print(f"  {direction}:")
        sorted_entries = sorted(directions[direction], key=lambda x: x["Время прибытия"])
        for entry in sorted_entries:
            print(f"    Время: {entry['Время прибытия'].strftime('%H:%M')}, Водитель: {entry['Водитель']}, Номер рейса: {entry['Номер рейса']}")

print(f"Всего сгенерированных рейсов: {len(schedule.trips)}")

Остановка: Улица маршала Тухачевского
  в сторону МЦД Панфиловская:
    Время: 06:00, Водитель: Вилен, Номер рейса: A0010AA
    Время: 06:20, Водитель: Валерий, Номер рейса: A003AA
    Время: 06:40, Водитель: Василевс, Номер рейса: A008AA
    Время: 07:00, Водитель: Прохор, Номер рейса: A001AA
    Время: 07:10, Водитель: Пересвет, Номер рейса: A001AA
    Время: 07:20, Водитель: Виталий, Номер рейса: A0021AA
    Время: 07:30, Водитель: Петр, Номер рейса: A0028AA
    Время: 07:40, Водитель: Платини, Номер рейса: A0022AA
    Время: 07:50, Водитель: Вадим, Номер рейса: A0013AA
    Время: 08:00, Водитель: Виктор, Номер рейса: A0020AA
    Время: 08:10, Водитель: Велимир, Номер рейса: A0018AA
    Время: 08:20, Водитель: Венедикт, Номер рейса: A0015AA
    Время: 08:30, Водитель: Вячеслав, Номер рейса: A001AA
    Время: 08:40, Водитель: Платон, Номер рейса: A0014AA
    Время: 08:50, Водитель: Президий, Номер рейса: A004AA
    Время: 09:00, Водитель: Прокл, Номер рейса: A007AA
    Время: 09:20, 